<a href="https://colab.research.google.com/github/yukinaga/ai_music/blob/main/section_3/03_performance_rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Performance RNN에 의한 작곡
 Performance RNN을 통해 표현력이 풍부한 피아노곡을 생성하세요.

## 라이브러리 설치
Magenta와 함께 음악 생성용 라이브러리 pyFluid Synth, MIDI 데이터를 처리하기 위한 pretty_midi 등을 설치합니다.

In [ ]:
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -qU pyfluidsynth pretty_midi
!pip install -qU magenta

## 기점이 되는 멜로디의 설정
RNN 의 기점이 되는 NoteSequence 를 설정합니다.  
너무 길게 하면 생성된 곡과의 위화감이 현저해지므로 짧게 설정합시다.


In [ ]:
import magenta
import note_seq
from note_seq.protobuf import music_pb2

seed = music_pb2.NoteSequence()  # NoteSequence

# notesにnoteを追加
seed.notes.add(pitch=80, start_time=0.0, end_time=0.4, velocity=80)
seed.notes.add(pitch=80, start_time=0.4, end_time=0.8, velocity=80)
seed.notes.add(pitch=87, start_time=0.8, end_time=1.2, velocity=80)
seed.notes.add(pitch=87, start_time=1.2, end_time=1.6, velocity=80)
seed.notes.add(pitch=89, start_time=1.6, end_time=2.0, velocity=80)
seed.notes.add(pitch=89, start_time=2.0, end_time=2.4, velocity=80)
seed.notes.add(pitch=87, start_time=2.4, end_time=3.2, velocity=80)

seed.total_time = 3.2  # 所要時間
seed.tempos.add(qpm=75);  # 曲のテンポを指定

note_seq.plot_sequence(seed)  # NoteSequenceの可視化
note_seq.play_sequence(seed, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## Performance RNN 초기화

Performacen RNN은 음의 강약이나 타이밍 등의 미묘한 표현까지 조정된 고도의 곡 생성을 가능하게 합니다.

https://github.com/magenta/magenta/tree/main/magenta/models/performance_rnn

학습완료된 모델은 Bundle 파일(.mag 파일)에 저장되어 있습니다.  
아래 코드는 학습된 모델 performance_with_dynamics.mag 을 읽고 곡 생성기를 설정하고 있습니다

In [ ]:
from magenta.models.performance_rnn import performance_sequence_generator
from magenta.models.shared import sequence_generator_bundle

# モデルの初期化
note_seq.notebook_utils.download_bundle("performance_with_dynamics.mag", "/models/")  # Bundle（.magファイル）をダウンロード
bundle = sequence_generator_bundle.read_bundle_file("/models/performance_with_dynamics.mag")  # Bundleの読み込み
generator_map = performance_sequence_generator.get_generator_map()
performance_rnn = generator_map["performance_with_dynamics"](checkpoint=None, bundle=bundle)  # 生成器の設定
performance_rnn.initialize()  # 初期化

## 곡 생성

각 설정을 실시한 후, 생성기에 의해 곡을 생성합니다.  
`temperature`를 변경함으로써 곡의 랜덤 정도를 조정할 수 있습니다.

In [ ]:
from note_seq.protobuf import generator_pb2

total_time = 180 # 曲の長さ（秒）
temperature = 1.0 # 曲の「ランダム度合い」を決める定数

base_end_time = max(note.end_time for note in seed.notes)  #ベース曲の終了時刻

# 生成器に関する設定
generator_options = generator_pb2.GeneratorOptions()  # 生成器のオプション
generator_options.args["temperature"].float_value = temperature  # ランダム度合い
generator_options.generate_sections.add(
    start_time=base_end_time,  # 作曲開始時刻
    end_time=total_time)  # 作曲終了時刻

# 曲の生成
gen_seq = performance_rnn.generate(seed, generator_options)

note_seq.plot_sequence(gen_seq)  # NoteSequenceの可視化
note_seq.play_sequence(gen_seq, synth=note_seq.fluidsynth)  # NoteSequenceの再生

## MIDI 파일 저장 및 다운로드
`노트Sequence`를 MIDI 데이터로 변환하여 저장하고 다운로드합니다.


In [ ]:
from google.colab import files

note_seq.sequence_proto_to_midi_file(gen_seq, "performance_rnn.mid")  #MIDI　データに変換し保存
files.download("performance_rnn.mid")  # ダウンロード